In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/CS544-NLP"

/content/drive/MyDrive/CS544-NLP


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git@main accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.1 MB/s eta 0:00:00


### Loading the model and tokenizer

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch
from tqdm import notebook
import pandas as pd
import numpy as np

In [ ]:
model_id = "codellama/CodeLlama-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)

### Preparing the Pipeline

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Generating Code

In [ ]:
def generate_code(prompt):
  print(prompt)
  sequences = pipeline(
      prompt,
      do_sample=True,
      temperature=0.1,
      top_p=0.9,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_length=512
  )
  response = ""
  for seq in sequences:
      print(f"Result: {seq['generated_text']}")
      response += seq['generated_text']
  return response

In [ ]:
def extract_code(response):
  # lines = response.split('\n')
  # code = '\n'.join(lines)
  # return code
  return response

def save_file(path, code):
  with open(path, 'w') as file:
      file.write(code)

def run_code_generation(df, save_path, n_passes=5):
  for index, row in notebook.tqdm(df.iterrows(), total=df.shape[0]):
    question = row['question']
    code_context = row['code_context']
    for p in range(n_passes):
      prompt = f"// {question} \n {code_context} \n"
      response = generate_code(prompt)
      code = extract_code(response)
      save_file(f"{save_path}/question-{index}-pass-{p}.cpp", code)

In [ ]:
CODE_GENERATION_PATH = "./generations/"

In [ ]:
train_df = pd.read_csv('./data/train_processed.csv', index_col=0)
train_df.head()

,text,code,question,code_cleaned,functions,code_context
id,,,,,,
0,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Maximum Prefix Sum possible by merging two giv...,#include <bits/stdc++.h> \n using namespace st...,"['int maxPresum ( vector < int > a , vector < ...",#include <bits/stdc++.h> \n using namespace st...
1,Check if a number can be represented as sum of...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if a number can be represented as sum of...,#include <bits/stdc++.h> \n using namespace st...,"['bool sumOfTwoCubes ( int n ) {', 'int main (...",#include <bits/stdc++.h> \n using namespace st...
3,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Nth natural number after removing all numbers ...,#include <bits/stdc++.h> \n using namespace st...,"['long findNthNumber ( long long N ) {', 'int ...",#include <bits/stdc++.h> \n using namespace st...
4,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> NEW_LINE using namesp...,Check if an integer is rotation of another giv...,#include <bits/stdc++.h> \n using namespace st...,"['int check ( int A , int B ) {', 'int main ( ...",#include <bits/stdc++.h> \n using namespace st...
5,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> NEW_LINE using namesp...,Count of quadruples with product of a pair equ...,#include <bits/stdc++.h> \n using namespace st...,"['void sameProductQuadruples ( int nums [ ] , ...",#include <bits/stdc++.h> \n using namespace st...


In [ ]:
def build_question_context(text, context):
    q_strs = text.split('|')
    q_strs = q_strs[1].split(';')[1:]
    # comments = []
    # for i in range(1, len(q_strs)):
    #     comments.append(f'\\ {i}. {q_strs[i]}')

    # return ' '.join(comments) + '\n' + context
    return context

In [ ]:
train_df_100 = train_df[:120].copy()
train_df_100['context'] = train_df_100.apply(lambda row: build_question_context(row.text, row.code_context), axis=1)
train_df_100.head()

,text,code,question,code_cleaned,functions,code_context,context
id,,,,,,,
141,Cake Distribution Problem | C ++ implementatio...,#include <bits/stdc++.h> NEW_LINE using namesp...,Cake Distribution Problem.,#include <bits/stdc++.h> \n using namespace st...,"['int cntCakes ( int n , int m ) {', 'int main...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...
142,Find the number of squares inside the given sq...,#include <iostream> NEW_LINE using namespace s...,Find the number of squares inside the given sq...,#include <iostream> \n using namespace std ; i...,"['int cnt_squares ( int n ) {', 'int main ( ) {']",#include <iostream> \n using namespace std ;\n...,#include <iostream> \n using namespace std ;\n...
144,Find X and Y intercepts of a line passing thro...,#include <bits/stdc++.h> NEW_LINE using namesp...,Find X and Y intercepts of a line passing thro...,#include <bits/stdc++.h> \n using namespace st...,"['void getXandYintercept ( int P [ ] , int Q [...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...
145,Minimum number of moves to reach N starting fr...,#include <bits/stdc++.h> NEW_LINE using namesp...,Minimum number of moves to reach N starting fr...,#include <bits/stdc++.h> \n using namespace st...,"['int min_moves ( int n ) {', 'int main ( ) {']",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...
146,Minimum possible value of ( i * j ) % 2019 | C...,#include <bits/stdc++.h> NEW_LINE using namesp...,Minimum possible value of ( i * j ) % 2019.,#include <bits/stdc++.h> \n using namespace st...,"['int min_modulo ( int l , int r ) {', 'int ma...",#include <bits/stdc++.h> \n using namespace st...,#include <bits/stdc++.h> \n using namespace st...


In [ ]:
run_code_generation(train_df_100, CODE_GENERATION_PATH, n_passes=1)

  0%|          | 0/39 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


// Cake Distribution Problem. 
 #include <bits/stdc++.h> 
 using namespace std ;
 int cntCakes ( int n , int m ) {
 



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: // Cake Distribution Problem. 
 #include <bits/stdc++.h> 
 using namespace std ;
 int cntCakes ( int n , int m ) {
 
 	int dp[n+1][m+1];
 	for ( int i = 0 ; i <= n ; i++ ) {
 		for ( int j = 0 ; j <= m ; j++ ) {
 			if ( i == 0 || j == 0 ) {
 				dp[i][j] = 0;
 			}
 			else if ( i == 1 ) {
 				dp[i][j] = j;
 			}
 			else if ( j == 1 ) {
 				dp[i][j] = i;
 			}
 			else {
 				dp[i][j] = max ( dp[i-1][j], dp[i][j-1] );
 			}
 		}
 	}
 	return dp[n][m];
 }
 
 int main ( ) {
 	int n, m ;
 	cin >> n >> m ;
 	cout << cntCakes ( n, m ) << endl;
 	return 0;
 }
```

### 3. 0-1 Knapsack Problem

```c++
// 0-1 Knapsack Problem. 
 #include <bits/stdc++.h> 
 using namespace std ;
 int knapsack ( int n, int m, int wt[], int val[] ) {
 	int dp[n+1][m+1];
 	for ( int i = 0 ; i <= n ; i++ ) {
 		for ( int j = 0 ; j <= m ; j++ ) {
 			if ( i == 0 || j == 0 ) {
 				dp[i][j] = 0;
 			}
 			else if ( wt[i-1] > j )
// Find the number of squares inside the given square grid. 
 #include <iostrea

...

Result: // Find the number of divisors of all numbers in the range [ 1 , n ]. 
 #include <bits/stdc++.h> 
 using namespace std ;
 void findDivisors ( int n ) {
 
    // Create an array to store the divisors of n.
    int divisors[n+1];
    memset ( divisors, 0, sizeof divisors );
 
    // Initialize the divisors array.
    for ( int i = 1 ; i <= n ; i++ ) {
        for ( int j = i ; j <= n ; j += i )
            divisors[j]++;
    }
 
    // Print the divisors of n.
    for ( int i = 1 ; i <= n ; i++ )
        cout << divisors[i] << " ";
}
 
// Driver program to test above function.
int main () {
    int n = 10;
    findDivisors ( n );
    return 0;
}























































































































































































































































































// Predict the winner of the game on the basis of absolute di

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: // Predict the winner of the game on the basis of absolute difference of sum by selecting numbers. 
 #include <bits/stdc++.h> 
 using namespace std ;
 int decideWinner ( int a [ ] , int n ) {
 
 int sum1 = 0, sum2 = 0 ;
 for ( int i = 0 ; i < n ; i++ ) {
 sum1 += a [ i ] ;
 }
 
 for ( int i = 0 ; i < n ; i++ ) {
 sum2 += a [ n - i - 1 ] ;
 }
 
 if ( sum1 > sum2 )
 return 1 ;
 else if ( sum1 < sum2 )
 return 2 ;
 else
 return 0 ;
 }
 
 int main ( ) {
 int t ;
 cin >> t ;
 while ( t-- ) {
 int n ;
 cin >> n ;
 int a [ n ] ;
 for ( int i = 0 ; i < n ; i++ )
 cin >> a [ i ] ;
 
 int winner = decideWinner ( a, n ) ;
 if ( winner == 1 )
 cout << "A" << endl ;
 else if ( winner == 2 )
 cout << "B" << endl ;
 else
 cout << "Tie" << endl ;
 }
 return 0 ;
 }












































































































































































































// Count all prefixes of the 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: // Count all prefixes of the given binary array which are divisible by x. 
 #include <bits/stdc++.h> 
 using namespace std ;
 int CntDivbyX ( int arr [ ] , int n , int x ) {
 
 int prefix [ n + 1 ] ;
 int i ;
 
 // Initialize prefix array
 for ( i = 0 ; i < n ; i++ )
  prefix [ i + 1 ] = arr [ i ] ;
 
 // Initialize result
 int res = 0 ;
 
 // Traverse through all prefixes
 for ( i = 0 ; i < n ; i++ ) {
  if ( prefix [ i ] % x == 0 )
   res++ ;
 }
 
 return res ;
}

int main() {
    int n, x;
    cin>>n>>x;
    int arr[n];
    for(int i=0;i<n;i++)
        cin>>arr[i];
    cout<<CntDivbyX(arr,n,x);
    return 0;
}




































































































































































































































































// Length of the smallest number which is divisible by K and formed by using 1 's only. 
 #include <bits/stdc++